
# Épreuve pratique NSI – Sujet 1 : *Cycle menstruel*

Notebook de solution complet : fonctions, tests et explications.



## Sommaire
1. [Question 1 — `est_bissextile`](#q1)
2. [Question 2 — `determiner_phase`](#q2)
3. [Question 3 — Tests de `ajouter_jours`](#q3)
4. [Question 4 — `calendrier_cycles` (format iCalendar)](#q4)



<a id="q1"></a>

## Question 1 — `est_bissextile`

Une année est bissextile si elle est :
- divisible par 4, **sauf** si elle est divisible par 100,  
- **sauf** si elle est aussi divisible par 400 (elle redevient bissextile).

Exemples : 2024 ✅, 2100 ❌, 2000 ✅.


In [1]:

def est_bissextile(annee: int) -> bool:
    """Renvoie True si l'année est bissextile, False sinon."""
    return (annee % 4 == 0 and annee % 100 != 0) or (annee % 400 == 0)


In [2]:

# Tests unitaires (Q1)
assert est_bissextile(2024) is True     # divisible par 4, pas par 100
assert est_bissextile(2100) is False    # divisible par 4 et 100 mais pas 400
assert est_bissextile(2000) is True     # divisible par 4, 100 et 400

print("Q1 ✅ Tous les tests passent.")


Q1 ✅ Tous les tests passent.



<a id="q2"></a>

## Question 2 — `determiner_phase`

Phases du cycle (jour de 1 à 28) :
- **Phase 1 (Règles)** : jours **1 à 5**
- **Phase 2 (Folliculaire)** : jours **6 à 13**
- **Phase 3 (Ovulation)** : jour **14**
- **Phase 4 (Lutéale)** : jours **15 à 28**

On ajoute une assertion pour garantir que `jour` est dans `[1, 28]`.


In [3]:

def determiner_phase(jour: int) -> int:
    """Renvoie le numéro de la phase du cycle pour un jour donné (1..28)."""
    assert 1 <= jour <= 28, "le jour doit être compris entre 1 et 28"
    if 1 <= jour <= 5:
        return 1
    elif 6 <= jour <= 13:
        return 2
    elif jour == 14:
        return 3
    else:
        return 4


In [4]:

# Tests unitaires (Q2)
assert determiner_phase(1) == 1
assert determiner_phase(5) == 1
assert determiner_phase(6) == 2
assert determiner_phase(13) == 2
assert determiner_phase(14) == 3
assert determiner_phase(15) == 4
assert determiner_phase(28) == 4

print("Q2 ✅ Tous les tests passent.")


Q2 ✅ Tous les tests passent.



<a id="q3"></a>

## Question 3 — Tests de `ajouter_jours`

On réutilise les fonctions utilitaires fournies et on ajoute **au moins trois tests pertinents**,
avec des cas limites : fin de mois, passage d'année, année bissextile.


In [5]:

import calendar

def jours_dans_mois(annee, mois):
    """Renvoie le nombre de jours dans un mois donné d'une année donnée.
       Utilise le module calendar pour gérer les années bissextiles."""
    if mois == 2:  # février
        return 29 if calendar.isleap(annee) else 28
    elif mois in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    else:
        return 30

def ajouter_jours(date, nb_jours):
    """Ajoute nb_jours à une date donnée et renvoie la nouvelle date.
       La date est représentée par un tuple (jour, mois, année)."""
    jour, mois, annee = date
    jour = jour + nb_jours

    # Ajustement du jour et du mois si dépassement
    while jour > jours_dans_mois(annee, mois):
        jour = jour - jours_dans_mois(annee, mois)
        mois = mois + 1
        if mois > 12:  # passage à l'année suivante
            mois = 1
            annee = annee + 1

    return (jour, mois, annee)


In [6]:

def test_ajouter_jours():
    # Cas fourni : ajout dans le même mois
    assert ajouter_jours((7, 9, 2025), 3) == (10, 9, 2025)  # contrôle de base
    
    # Cas 1 — Fin de mois de 30 jours -> mois suivant
    # Justification : vérifier l'ajustement lorsque l'on dépasse un mois de 30 jours (septembre).
    assert ajouter_jours((30, 9, 2025), 2) == (2, 10, 2025)
    
    # Cas 2 — Passage d'année
    # Justification : vérifier l'incrément de l'année lorsque l'on dépasse décembre.
    assert ajouter_jours((31, 12, 2025), 1) == (1, 1, 2026)
    
    # Cas 3 — Février bissextile
    # Justification : vérifier que février contient 29 jours en année bissextile.
    assert ajouter_jours((28, 2, 2024), 1) == (29, 2, 2024)

test_ajouter_jours()
print("Q3 ✅ Tous les tests passent.")


Q3 ✅ Tous les tests passent.



<a id="q4"></a>

## Question 4 — `calendrier_cycles` (format iCalendar)

### Problème identifié
Dans la version initiale, les dates `DTSTART` étaient construites via `str(annee)+str(mois)+str(jour)`,
ce qui ne garantit **pas** le format `AAAAMMJJ` (8 chiffres) lorsque `mois` ou `jour` est inférieur à 10.

### Correction
Formater la date avec des zéros non significatifs : `f"{annee:04d}{mois:02d}{jour:02d}"`.

On génère tous les débuts de règles dans les **100 jours suivants** (date incluse) avec un **cycle de 28 jours**.


In [7]:

def calendrier_cycles(date_regles):
    """Renvoie une chaîne au format iCalendar listant les dates de début de règles
    dans les 100 jours suivant `date_regles` (date incluse), pour un cycle de 28 jours.
    """
    cal_lignes = ['BEGIN:VCALENDAR', 'VERSION:2.0', 'PRODID:Cycle menstruel']
    date_courante = date_regles
    jours_ecoules = 0

    # On ajoute la date courante tant qu'on reste dans la fenêtre des 100 jours
    while jours_ecoules <= 100:
        jour, mois, annee = date_courante
        cal_lignes.append('BEGIN:VEVENT')
        cal_lignes.append('SUMMARY:Règles')
        date = f"{annee:04d}{mois:02d}{jour:02d}"
        cal_lignes.append('DTSTART:' + date)
        cal_lignes.append('END:VEVENT')
        date_courante = ajouter_jours(date_courante, 28)
        jours_ecoules += 28

    cal_lignes.append('END:VCALENDAR')
    return '\n'.join(cal_lignes)


In [8]:

# Démonstration : génération d'un calendrier à partir du 12/03/2026
cal_sample = calendrier_cycles((12, 3, 2026))
print(cal_sample)


BEGIN:VCALENDAR
VERSION:2.0
PRODID:Cycle menstruel
BEGIN:VEVENT
SUMMARY:Règles
DTSTART:20260312
END:VEVENT
BEGIN:VEVENT
SUMMARY:Règles
DTSTART:20260409
END:VEVENT
BEGIN:VEVENT
SUMMARY:Règles
DTSTART:20260507
END:VEVENT
BEGIN:VEVENT
SUMMARY:Règles
DTSTART:20260604
END:VEVENT
END:VCALENDAR



> **Optionnel (si la bibliothèque `ics` est installée)** : valider le contenu produit.
Exécutez la cellule ci-dessous après avoir installé `ics` avec `pip install ics`.


In [16]:

from ics import Calendar
cal = Calendar(cal_sample)
print("Nombre d'événements lus par ics :", len(cal.events))

Nombre d'événements lus par ics : 4
